<a href="https://colab.research.google.com/github/thilanC/DataScience_assignment/blob/main/Data_Science_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [3]:
#import data set to workbook as df
df=pd.read_csv('/content/drive/MyDrive/Machine learning/DS_CaseStudy_Churn_Pred_Data.csv')

In [4]:
#check data frame data types
df.dtypes

delay                 float64
created_date            int64
pickup_date           float64
distance              float64
driver_rating         float64
fare                  float64
discount              float64
payment_method        float64
promocode              object
taximodelid             int64
travel_status           int64
trip_fare             float64
profile_created         int64
status_description     object
pre_booked              int64
passengerid             int64
tripid                  int64
dtype: object

In [5]:
#check df shape (number of raws and columns)
df.shape

(9004901, 17)

In [6]:
#Get sum of null values
df.isna().sum()

delay                 5077799
created_date                0
pickup_date           5077799
distance              5081668
driver_rating         9004901
fare                  5081668
discount              5081668
payment_method              7
promocode             7063769
taximodelid                 0
travel_status               0
trip_fare             5081668
profile_created             0
status_description          0
pre_booked                  0
passengerid                 0
tripid                      0
dtype: int64

In [7]:
#Dataframe 1st 5 raws
df.head()

,delay,created_date,pickup_date,distance,driver_rating,fare,discount,payment_method,promocode,taximodelid,travel_status,trip_fare,profile_created,status_description,pre_booked,passengerid,tripid
0,NaN,20190809,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1,12,NaN,20170223,Unavailable,0,107206,7542464
1,8.38,20190701,20190701.0,16.49,NaN,676.13,0.0,1.0,NaN,1,1,676.13,20180201,Completed,0,123034,6044316
2,16.45,20190830,20190830.0,14.39,NaN,836.63,0.0,1.0,NaN,2,1,836.63,20180905,Completed,0,201136,1959290
3,NaN,20190709,NaN,NaN,NaN,NaN,NaN,1.0,lpp42,1,12,NaN,20180113,Unavailable,0,403293,1250879
4,NaN,20190718,NaN,NaN,NaN,NaN,NaN,1.0,abac,4,12,NaN,20190702,Unavailable,0,448530,884375


In [8]:
df['promocode'].value_counts()

LPP53                     41852
AVG58                     35312
DC26                      35306
7T7EVEN                   31519
6to6even                  28582
                          ...  
6t06PROMO Removed             1
LLP50 Removed                 1
lPP62                         1
768110909 Removed             1
mount mary inn Removed        1
Name: promocode, Length: 3584, dtype: int64

In [9]:
df['status_description'].value_counts()

Completed                           3923005
Unavailable                         2585817
Passenger Cancel Before Accept      1376812
Cancel By Passenger After Accept    1012955
Driver & Dispatcher Cancellation      85667
Concurrent Request                    20366
Rejected                                279
Name: status_description, dtype: int64

In [10]:
#drop unnecessary attributes
df.drop(['pickup_date','driver_rating','payment_method','promocode','trip_fare','status_description','pre_booked','tripid'],axis=1,inplace=True)

In [11]:
df.head()

,delay,created_date,distance,fare,discount,taximodelid,travel_status,profile_created,passengerid
0,NaN,20190809,NaN,NaN,NaN,1,12,20170223,107206
1,8.38,20190701,16.49,676.13,0.0,1,1,20180201,123034
2,16.45,20190830,14.39,836.63,0.0,2,1,20180905,201136
3,NaN,20190709,NaN,NaN,NaN,1,12,20180113,403293
4,NaN,20190718,NaN,NaN,NaN,4,12,20190702,448530


In [12]:
#check how many customer in here
len(df.passengerid.unique())

518744

In [13]:
# Function to convert the integer date format to "YYYY-MM-DD" string
def convert_date_format(date_int):
    date_str = str(date_int)
    original_date = datetime.strptime(date_str, '%Y%m%d')
    formatted_date_string = original_date.strftime('%Y-%m-%d')
    return formatted_date_string

In [ ]:
# Apply the function to the date column
df['created_date'] = df['created_date'].apply(convert_date_format)
df['profile_created'] = df['profile_created'].apply(convert_date_format)

In [ ]:
df.head()

In [ ]:
#drop null value contaning raws in distance column
df.dropna(subset=['distance'], inplace=True)

 Here distance culumn has huge null values. That null value means zero distance or else. I take only traveled customer details. So here I made an assumption consider only traveled customer details, customer will only churn or not if they travel.

In [ ]:
df.shape

In [ ]:
#Get sum of null values
df.isna().sum()

In [ ]:
#fill remain null values using zero
df = df.fillna(0)

In [ ]:
# Count the number of trips for each 'passengerid' and create a num_trips
num_trips = df['passengerid'].value_counts().reset_index()

# Rename the columns to be more meaningfull
num_trips.columns = ['passengerid', 'Trip_count']

In [ ]:
num_trips.shape

In [ ]:
# Merge DataFrames num_trips and df on the 'passengerid' column
result = num_trips.merge(df[['passengerid', 'profile_created']], on='passengerid', how='left')
#drop duplicate values
result=result.drop_duplicates()

In [ ]:
result

Create new dataframe using **df** columns, it has trips and trip details of customers. so here I create new dataframe using given details.

In [ ]:
# Group by 'passengerid' and calculate the mean of the 'delay' column for each group
delays = df.groupby('passengerid')['delay'].agg('mean').reset_index()

# Rename the columns in the result DataFrame to be more meaningful
delays.columns = ['passengerid', 'delay']

# Merge DataFrames 'result' and 'delays' on the 'passengerid' column
result = result.merge(delays[['passengerid', 'delay']], on='passengerid', how='left')
# Remove any duplicate rows in the 'result' DataFrame
result=result.drop_duplicates()

In [ ]:
# Group by 'passengerid' and calculate the mean of the 'distance' column for each group
distance = df.groupby('passengerid')['distance'].agg('mean').reset_index()

# Rename the columns in the result DataFrame to be more meaningful
distance.columns = ['passengerid', 'distance']

# Merge DataFrames 'result' and 'distance' on the 'passengerid' column
result = result.merge(distance[['passengerid', 'distance']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Group by 'passengerid' and calculate the mean of the 'fare' column for each group
fare = df.groupby('passengerid')['fare'].agg('mean').reset_index()

# Rename the columns in the result DataFrame to be more meaningful
fare.columns = ['passengerid', 'fare']

# Merge DataFrames 'result' and 'fare' on the 'passengerid' column
result = result.merge(fare[['passengerid', 'fare']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Group by 'passengerid' and calculate the mean of the 'taximodelid' column for each group
taximodelid = df.groupby('passengerid')['taximodelid'].agg('mean').reset_index()

# Rename the columns in the result DataFrame to be more meaningful
taximodelid.columns = ['passengerid', 'taximodelid']

# Merge DataFrames 'result' and 'taximodelid' on the 'passengerid' column
result = result.merge(taximodelid[['passengerid', 'taximodelid']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Group by 'passengerid' and calculate the mean of the 'travel_status' column for each group
travel_status = df.groupby('passengerid')['travel_status'].agg('mean').reset_index()

# Rename the columns in the result DataFrame to be more meaningful
travel_status.columns = ['passengerid', 'travel_status']

# Merge DataFrames 'result' and 'travel_status' on the 'passengerid' column
result = result.merge(travel_status[['passengerid', 'travel_status']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Convert the 'profile_created' column to datetime
result['profile_created'] = pd.to_datetime(result['profile_created'])

# Group by 'passengerid' and find the latest 'created_date' for each group
latest_dates = df.groupby('passengerid')['created_date'].agg('max').reset_index()

# Rename the columns in the 'latest_dates' DataFrame
latest_dates.columns = ['passengerid', 'Latest_Date']

# Merge the 'result' DataFrame with the 'latest_dates' DataFrame on the 'passengerid' column
result = result.merge(latest_dates[['passengerid', 'Latest_Date']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
result

In [ ]:
result['Latest_Date'] = pd.to_datetime(result['Latest_Date'])

# Define a function to calculate the average value
def calculate_average(row):
    return (row['Latest_Date'] - row['profile_created']).days / row['Trip_count']

# Apply the function to each row and create a new 'average' column
result['average_days'] = result.apply(calculate_average, axis=1)

In [ ]:
result

In [ ]:
#here is the latest date of data set
result.Latest_Date.max()

In [ ]:
# Convert the 'date_column' to datetime
df['created_date'] = pd.to_datetime(df['created_date'])

In [ ]:
# Define the date range
start_date = pd.to_datetime('2019-08-17')
end_date = pd.to_datetime('2019-08-31')

# Filter the DataFrame 'df' to include only dates within the specified date range
# Group the filtered data by 'passengerid' and count the dates in the specified date range for each group
date_counts1 = df[(df['created_date'] >= start_date) & (df['created_date'] <= end_date)].groupby('passengerid')['created_date'].count().reset_index()

# Rename the columns in the result DataFrame to make them more meaningful
date_counts1.columns = ['passengerid', 'Within_14days_trips']

# The 'date_counts' DataFrame now contains the count of trips within the specified 14-day date range for each unique passenger ID
print(date_counts1)

In [ ]:
# Merge the 'result' DataFrame with the 'date_counts' DataFrame based on the 'passengerid' column
result = result.merge(date_counts1[['passengerid', 'Within_14days_trips']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Define the date range
start_date = pd.to_datetime('2019-07-31')
end_date = pd.to_datetime('2019-08-31')

# Filter the DataFrame 'df' to include only dates within the specified date range
# Group the filtered data by 'passengerid' and count the dates in the specified date range for each group
date_counts2 = df[(df['created_date'] >= start_date) & (df['created_date'] <= end_date)].groupby('passengerid')['created_date'].count().reset_index()

# Rename the columns in the result DataFrame to make them more meaningful
date_counts2.columns = ['passengerid', 'Within_month_trips']

In [ ]:
# Merge the 'result' DataFrame with the 'date_counts' DataFrame based on the 'passengerid' column
result = result.merge(date_counts2[['passengerid', 'Within_month_trips']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
# Define the date range
start_date = pd.to_datetime('2019-06-30')
end_date = pd.to_datetime('2019-08-31')

# Filter the DataFrame 'df' to include only dates within the specified date range
# Group the filtered data by 'passengerid' and count the dates in the specified date range for each group
date_counts3 = df[(df['created_date'] >= start_date) & (df['created_date'] <= end_date)].groupby('passengerid')['created_date'].count().reset_index()

# Rename the columns in the result DataFrame
date_counts3.columns = ['passengerid', 'Within_2month_trips']

In [ ]:
result = result.merge(date_counts3[['passengerid', 'Within_2month_trips']], on='passengerid', how='left')
result=result.drop_duplicates()

In [ ]:
result

In [ ]:
#check sum of null values contain in each column
result.isna().sum()

In [ ]:
#Here we can fill null values using 0, here null values means that custormer did not travel withing that time period.

# Fill null values in the 'result' DataFrame with 0
result = result.fillna(0)

In [ ]:
result.isna().sum()

In [ ]:
# Creating boxplot
plt.boxplot(result['average_days'])
plt.show()

#here I can get a idea outliers in average_days column and set cutoff limit

In [ ]:
#get maximum average_days column value
result.average_days.max()

Here I define my custormer churn criteria,  

*   average_days - The average number of days customers take to travel
*   Within_month_trips - Customer's last month's travel frequency
* Within_2month_trips - Customer's last 2 month's travel frequency


***If the 'average_days' is less than 30 and there are no trips within the last month (Within_month_trips is 0) or if the 'average_days' is less than 60 and there are no trips within the last 2 months (Within_2month_trips is 0), the 'Churn' value is set to 'yes'; otherwise, it's set to 'no'.***

***if the 'average_days' is greater than 3 months and sets 'Churn' to 'yes' for customers with such long 'average_days'.***

In [ ]:
# Calculate the 'Churn' column based on specified conditions
result['Churn'] = np.where(
    ((result['average_days'] < 30) & (result['Within_month_trips'] == 0)) |
    ((result['average_days'] < 60) & (result['Within_2month_trips'] == 0)),
    'Yes',
    'No'
)

# Set 'Churn' to 'yes' for customers with 'average_days' greater than 270
result.loc[result['average_days'] > 270, 'Churn'] = 'Yes'


In [ ]:
result

In [ ]:
#calculate churn column values
result["Churn"].value_counts()

In [ ]:
result.head()

In [ ]:
# Create a new DataFrame 'df2' by dropping selected columns from the 'result' DataFrame
df2=result.drop(['profile_created','Latest_Date','Within_14days_trips','Within_month_trips','Within_2month_trips','passengerid'],axis=1)

In [ ]:
df2.head()

In [ ]:
df2_numeric = df2.drop(['Churn'],axis=1)

In [ ]:
#Create a figure and subplots
fig, axes = plt.subplots(2, 4, figsize=(12, 6))  # Adjust the size as needed

# List of column names you want to plot (assuming 7 columns)
columns_to_plot = df2_numeric.columns

# Loop through the columns and create individual boxplots
for i, col in enumerate(columns_to_plot):
    ax = axes[i // 4, i % 4]  # Determine the subplot position
    df2_numeric.boxplot(column=col, ax=ax)
    ax.set_title(col)

# Adjust layout and spacing
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
df2_numeric.loc[df2_numeric['distance']>=5000,'distance']=df2_numeric.distance.mean()
#df2_numeric.loc[df2_numeric['Fare']>=100,'Fare']=100

In [ ]:
#Create a figure and subplots
fig, axes = plt.subplots(2, 4, figsize=(12, 6))  # Adjust the size as needed

# List of column names you want to plot (assuming 7 columns)
columns_to_plot = df2_numeric.columns

# Loop through the columns and create individual boxplots
for i, col in enumerate(columns_to_plot):
    ax = axes[i // 4, i % 4]  # Determine the subplot position
    df2_numeric.boxplot(column=col, ax=ax)
    ax.set_title(col)

# Adjust layout and spacing
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
sns.heatmap(df2_numeric.corr(), annot=True)

In [ ]:
# Separate features and target variable
X = df2.drop('Churn', axis=1)
y = df2['Churn']

In [ ]:
# Normalize numerical variables
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
y.replace('Yes',1,inplace=True)
y.replace('No',0,inplace=True)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Handle class imbalance (e.g., using SMOTE)
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
log_model = LogisticRegression(max_iter=10000)
log_model.fit(X_train, y_train)

log_pred = log_model.predict(X_test)

print(classification_report(y_test, log_pred))
print(confusion_matrix(y_test, log_pred))

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)

print(classification_report(y_test, rfc_pred))
print(confusion_matrix(y_test, rfc_pred))

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

model_pred = model.predict(X_test)

print(classification_report(y_test, model_pred))
print(confusion_matrix(y_test, model_pred))

In [ ]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

model_pred = model.predict(X_test)

print(classification_report(y_test, model_pred))
print(confusion_matrix(y_test, model_pred))